**Adrián Yared Armas de la Nuez**

In [1]:
# Imports
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB, ComplementNB, CategoricalNB
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.metrics import accuracy_score

# Iris data Load
data = load_iris()
X, y = data.data, data.target

# Train and test data split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Naive Bayes Models
models = {
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB(),
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
}

# Para CategoricalNB, data discretization
categorical_encoder = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='uniform')
X_categorical = categorical_encoder.fit_transform(X)
models["CategoricalNB"] = CategoricalNB()

# Results
results = {"Model": [], "Without Cross-Validation": [], "With Cross-Validation": []}

for name, model in models.items():
    # with NO cross validation
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    # with cross validation
    if name == "CategoricalNB":
        cv_scores = cross_val_score(model, X_categorical, y, cv=5)
    else:
        cv_scores = cross_val_score(model, X, y, cv=5)

    # Save results
    results["Model"].append(name)
    results["Without Cross-Validation"].append(accuracy)
    results["With Cross-Validation"].append(cv_scores.mean())

# Show results
import pandas as pd
results_df = pd.DataFrame(results)
print(results_df)


           Model  Without Cross-Validation  With Cross-Validation
0     GaussianNB                  1.000000               0.953333
1  MultinomialNB                  0.900000               0.953333
2    BernoulliNB                  0.300000               0.333333
3   ComplementNB                  0.700000               0.666667
4  CategoricalNB                  0.966667               0.940000


# Justificación

Los resultados obtenidos parecen razonables y están alineados con el comportamiento esperado de los diferentes modelos Naive Bayes al aplicarse al conjunto de datos Iris.

## GaussianNB
- Trabaja bien con datos continuos y distribuciones gaussianas.
- Resultados esperados: Una precisión alta (generalmente >90%), tanto con como sin validación cruzada.
- Los resultados para GaussianNB (1.0 sin CV, 0.9533 con CV) son consistentes y razonables.

## MultinomialNB

- Este modelo es más adecuado para datos discretos o de conteo
- Una precisión algo menor que GaussianNB
- Los resultados (0.9 sin CV, 0.9533 con CV) sugieren que MultinomialNB se beneficia de la validación cruzada. Esto es plausible, ya que la CV suaviza posibles sesgos de la partición inicial.

## BernoulliNB
- Este modelo es más adecuado para datos binarios
- Precisión significativamente baja
- Los resultados (0.3 sin CV, 0.3333 con CV) reflejan este comportamiento esperado

## ComplementNB
- Es una variante de MultinomialNB diseñada para clases desbalanceadas
- Un desempeño intermedio, mejor que BernoulliNB pero posiblemente inferior a GaussianNB
- Los resultados (0.7 sin CV, 0.6667 con CV) son razonables para este caso

## CategoricalNB
- Este modelo está diseñado específicamente para datos categóricos. Para usarlo con Iris, discretizar los datos es necesario (como hice en el código con KBinsDiscretizer)
- Una precisión muy alta, pero algo menor que GaussianNB, ya que los datos de Iris no son naturalmente categóricos, que es la especialidad de este
- Los resultados (0.9667 sin CV, 0.94 con CV) son consistentes y razonables si los datos fueron discretizados correctamente